## Geocoding and directions for tourist attractions in Austria

##### The workflow order is as follows:
1. Import libraries
2. Import addresses
3. Geocode addresses
4. Plot addresses
5. Select start and end point to plot route

In [6]:
import openrouteservice as ors
from openrouteservice.directions import directions
import folium
import geopy
import leafmap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io

In [ ]:
# pip install openrouteservice

  Using cached openrouteservice-2.3.3-py3-none-any.whl (33 kB)
Note: you may need to restart the kernel to use updated packages.


##### API Key

In [31]:
# import config
# performs requests to the ORS API services
# client will be used in all examples
# client = ors.Client(key=config.api_key)

client = ors.Client(key='5b3ce3597851110001cf624889ce32dc9e8741dca14bd53556e3a687')

#### Import addresses

Import addresses in CSV file format 

In [8]:
upload = widgets.FileUpload(
        accept='*.csv',
        multiple=False)
display(upload)

FileUpload(value={}, accept='*.csv', description='Upload')

#### Data Cleaning 
Inspect the imported data and optimize it for geocoding

In [11]:
input_file = list(upload.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
df = pd.read_csv(content)
df.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44


In [16]:
# Merge street, zip and city fields into an address field Using DataFrame.apply() and lambda function

# df["address"] = df["street"].map(str) + ', ' + df["zip"].map(str)+ ', ' + df["city"]+ ', ' +  df['country']
# df.head()

df["address"] = df["Address1"].map(str) + ', ' + df["Address3"].map(str)+ ', ' + df["Address4"] #+ ', ' +  df['country']
df.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,address
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM"
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47, 114 39, STOCKHOLM"
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM"
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen, 111 57, STOCKHOLM"
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM"


#### Geocode addresses
Geocode addresses using geopy library and Nominatim

In [13]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [14]:
geolocator = Nominatim(user_agent="Practice")

In [17]:
locator = Nominatim(user_agent="Practice")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [18]:
df.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,address,location,point
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM","(Karlaplan, Östermalm, Östermalms stadsdelsomr...","(59.337219, 18.0908107, 0.0)"
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47, 114 39, STOCKHOLM","(47, Nybrogatan, Villastaden, Östermalm, Öster...","(59.3372072, 18.0790982, 0.0)"
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM","(Karlavägen, Östermalm, Östermalms stadsdelsom...","(59.3361757, 18.0973562, 0.0)"
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen, 111 57, STOCKHOLM","(Hötorgshallen, Hötorget, Klara, Norrmalm, Nor...","(59.3343273, 18.0626039, 0.0)"
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM","(Drottninggatan, Klara, Norrmalm, Norrmalms st...","(59.3357611, 18.0595426, 0.0)"


Drop any colummns with null coordinate values

In [20]:
df1 = df.dropna()
df1.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,address,location,point
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM","(Karlaplan, Östermalm, Östermalms stadsdelsomr...","(59.337219, 18.0908107, 0.0)"
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM","(Karlavägen, Östermalm, Östermalms stadsdelsom...","(59.3361757, 18.0973562, 0.0)"
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM","(Drottninggatan, Klara, Norrmalm, Norrmalms st...","(59.3357611, 18.0595426, 0.0)"
5,Butik,114,PK-Huset,Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms län,08/796 98 10,"Norrlandsgatan 3, 111 47, STOCKHOLM","(3, Norrlandsgatan, Klara, Norrmalm, Norrmalms...","(59.3331769, 18.0712808, 0.0)"
7,Butik,132,Marieberg,Rålambsvägen 7-9,112 59,STOCKHOLM,Stockholms län,08/13 30 95,"Rålambsvägen 7-9, 112 59, STOCKHOLM","(Rålambsvägen, Marieberg, Kungsholmens stadsde...","(59.3284404, 18.0177153, 0.0)"


In [21]:
df.shape

(10, 11)

In [22]:
# split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df.index)
df1.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,address,location,point,latitude,longitude,altitude
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM","(Karlaplan, Östermalm, Östermalms stadsdelsomr...","(59.337219, 18.0908107, 0.0)",59.337219,18.090811,0.0
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM","(Karlavägen, Östermalm, Östermalms stadsdelsom...","(59.3361757, 18.0973562, 0.0)",59.336176,18.097356,0.0
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM","(Drottninggatan, Klara, Norrmalm, Norrmalms st...","(59.3357611, 18.0595426, 0.0)",59.335761,18.059543,0.0
5,Butik,114,PK-Huset,Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms län,08/796 98 10,"Norrlandsgatan 3, 111 47, STOCKHOLM","(3, Norrlandsgatan, Klara, Norrmalm, Norrmalms...","(59.3331769, 18.0712808, 0.0)",59.333177,18.071281,0.0
7,Butik,132,Marieberg,Rålambsvägen 7-9,112 59,STOCKHOLM,Stockholms län,08/13 30 95,"Rålambsvägen 7-9, 112 59, STOCKHOLM","(Rålambsvägen, Marieberg, Kungsholmens stadsde...","(59.3284404, 18.0177153, 0.0)",59.328440,18.017715,0.0


In [24]:
df1 = df1.drop(['Address1', 'Address3', 'Address4', 'Address5'], axis=1)

In [26]:
df1.head()

,Typ,Nr,Namn,Telefon,address,location,point,latitude,longitude,altitude
0,Butik,102,Fältöversten,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM","(Karlaplan, Östermalm, Östermalms stadsdelsomr...","(59.337219, 18.0908107, 0.0)",59.337219,18.090811,0.0
2,Butik,106,Garnisonen,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM","(Karlavägen, Östermalm, Östermalms stadsdelsom...","(59.3361757, 18.0973562, 0.0)",59.336176,18.097356,0.0
4,Butik,113,Sergel,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM","(Drottninggatan, Klara, Norrmalm, Norrmalms st...","(59.3357611, 18.0595426, 0.0)",59.335761,18.059543,0.0
5,Butik,114,PK-Huset,08/796 98 10,"Norrlandsgatan 3, 111 47, STOCKHOLM","(3, Norrlandsgatan, Klara, Norrmalm, Norrmalms...","(59.3331769, 18.0712808, 0.0)",59.333177,18.071281,0.0
7,Butik,132,Marieberg,08/13 30 95,"Rålambsvägen 7-9, 112 59, STOCKHOLM","(Rålambsvägen, Marieberg, Kungsholmens stadsde...","(59.3284404, 18.0177153, 0.0)",59.328440,18.017715,0.0


#### Plot addresses
Plot geocoded addresses using folium and leafmap mapping libraries

In [27]:
map1 = folium.Map(
    location=[50.1109,8.6821],
    tiles='openstreetmap',
    zoom_start=6,
)

df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [28]:
Map = leafmap.Map(center=(50.1109,8.6821), zoom=5)
Map.add_xy_data(df1, x="longitude", y="latitude", layer_name="restaurants")
Map

Map(center=[50.1109, 8.6821], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

#### Routing from start point to end point
Implement routing using openroutes service library

# widgets
start_dropdown = widgets.Dropdown(options=list(df.address),
                                     value=list(df.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df.address),
                                   value=list(df.address)[1],
                                   description='end:',
                                   disabled=False)

# function
# calculate distance from a to b
# display map with ant path from a to b
def get_coordinates(a, b):
    start_lat = df[df.address == a].latitude.values[0]
    start_long = df[df.address == a].longitude.values[0]
    end_lat = df[df.address == b].latitude.values[0]
    end_long = df[df.address == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
    
    m = Map(center=(50.1109,8.6821), zoom=5, scroll_wheel_zoom=True)
    ant_path = AntPath(locations=[start, end])
    m.add_layer(ant_path)
    measure = MeasureControl(position='topleft', active_color='red', primary_length_unit='miles')
    m.add_control(measure)
    display(m)
    
    return distance.distance(start, end).miles

# interaction between function and widget
widgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

In [36]:
df1

,Typ,Nr,Namn,Telefon,address,location,point,latitude,longitude,altitude
0,Butik,102,Fältöversten,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM","(Karlaplan, Östermalm, Östermalms stadsdelsomr...","(59.337219, 18.0908107, 0.0)",59.337219,18.090811,0.0
2,Butik,106,Garnisonen,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM","(Karlavägen, Östermalm, Östermalms stadsdelsom...","(59.3361757, 18.0973562, 0.0)",59.336176,18.097356,0.0
4,Butik,113,Sergel,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM","(Drottninggatan, Klara, Norrmalm, Norrmalms st...","(59.3357611, 18.0595426, 0.0)",59.335761,18.059543,0.0
5,Butik,114,PK-Huset,08/796 98 10,"Norrlandsgatan 3, 111 47, STOCKHOLM","(3, Norrlandsgatan, Klara, Norrmalm, Norrmalms...","(59.3331769, 18.0712808, 0.0)",59.333177,18.071281,0.0
7,Butik,132,Marieberg,08/13 30 95,"Rålambsvägen 7-9, 112 59, STOCKHOLM","(Rålambsvägen, Marieberg, Kungsholmens stadsde...","(59.3284404, 18.0177153, 0.0)",59.328440,18.017715,0.0
12,Butik,140,Torsplan,08/31 73 49,"Solnavägen 2b, 113 65, STOCKHOLM","(2B, Solnavägen, Röda bergen, Vasastaden, Norr...","(59.3464104, 18.0332565, 0.0)",59.346410,18.033257,0.0
13,Butik,143,Odenplan,08/32 35 29,"Odengatan 58, 113 22, STOCKHOLM","(58, Odengatan, Sibirien, Vasastaden, Norrmalm...","(59.3434945, 18.0528016, 0.0)",59.343494,18.052802,0.0
14,Butik,144,Jarlaplan,08/24 31 15,"Birger Jarlsgatan 84, 114 20, STOCKHOLM","(84, Birger Jarlsgatan, Lärkstaden, Östermalm,...","(59.3446986, 18.063395, 0.0)",59.344699,18.063395,0.0
19,Butik,166,Söderhallarna,08/641 86 75,"Medborgarplatsen 3, 118 26, STOCKHOLM","(3, Medborgarplatsen, Södermalm, Södermalms st...","(59.3144846, 18.0712026, 0.0)",59.314485,18.071203,0.0
23,Butik,173,Hammarby Sjöstad,08/640 00 44,"Lugnets Allé 26-28, 120 66, STOCKHOLM","(26-28, Lugnets Allé, Södra Hammarbyhamnen, Sö...","(59.3023089, 18.1034195, 0.0)",59.302309,18.103420,0.0


In [35]:
start_dropdown = widgets.Dropdown(options=list(df1.address),
                                     value=list(df1.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df1.address),
                                   value=list(df1.address)[1],
                                   description='end:',
                                   disabled=False)


print('start_dropdown',start_dropdown)
print('end_dropdown',end_dropdown)


def get_coordinates(df1):
    start_lat = df1[df1.address == a].latitude.values[0]
    start_long = df1[df1.address == a].longitude.values[1]
    end_lat = df1[df1.address == b].latitude.values[0]
    end_long = df1[df1.address == b].longitude.values[1]
    start, end = (start_lat, start_long), (end_lat, end_long)

    

abc = get_coordinates(df1)
print(abc.start())




# # interaction between function and widget
# widgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

# # directions
# route = client.directions(coordinates=coordinates,
#                           profile='driving-car',
#                           format='geojson')

# # map
# map_directions = folium.Map(location=[50.1109,8.6821], zoom_start=5)

# # add geojson to map
# folium.GeoJson(route, name='route').add_to(map_directions)

# # add layer control to map (allows layer to be turned on or off)
# folium.LayerControl().add_to(map_directions)

# # display map
# map_directions

start_dropdown Dropdown(description='start:', options=('Karlaplan 13, 115 20, STOCKHOLM', 'Karlavägen 100 A, 115 26, STOCKHOLM', 'Drottninggatan 45, 111 21, STOCKHOLM', 'Norrlandsgatan 3, 111 47, STOCKHOLM', 'Rålambsvägen 7-9, 112 59, STOCKHOLM', 'Solnavägen 2b, 113 65, STOCKHOLM', 'Odengatan 58, 113 22, STOCKHOLM', 'Birger Jarlsgatan 84, 114 20, STOCKHOLM', 'Medborgarplatsen 3, 118 26, STOCKHOLM', 'Lugnets Allé 26-28, 120 66, STOCKHOLM'), value='Karlaplan 13, 115 20, STOCKHOLM')
end_dropdown Dropdown(description='end:', index=1, options=('Karlaplan 13, 115 20, STOCKHOLM', 'Karlavägen 100 A, 115 26, STOCKHOLM', 'Drottninggatan 45, 111 21, STOCKHOLM', 'Norrlandsgatan 3, 111 47, STOCKHOLM', 'Rålambsvägen 7-9, 112 59, STOCKHOLM', 'Solnavägen 2b, 113 65, STOCKHOLM', 'Odengatan 58, 113 22, STOCKHOLM', 'Birger Jarlsgatan 84, 114 20, STOCKHOLM', 'Medborgarplatsen 3, 118 26, STOCKHOLM', 'Lugnets Allé 26-28, 120 66, STOCKHOLM'), value='Karlavägen 100 A, 115 26, STOCKHOLM')


NameError: name 'a' is not defined

In [ ]:
start_dropdown = widgets.Dropdown(options=list(df.address),
                                     value=list(df.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df.address),
                                   value=list(df.address)[1],
                                   description='end:',
                                   disabled=False)

def get_coordinates(a, b):
    start_lat = df[df.address == a].latitude.values[0]
    start_long = df[df.address == a].longitude.values[0]
    end_lat = df[df.address == b].latitude.values[0]
    end_long = df[df.address == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
    coordinates = [[start_lat, start_long], [end_lat, end_long]]

    route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')


    map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)
    folium.GeoJson(route, name='route').add_to(map_directions)

    folium.LayerControl().add_to(map_directions)

    map_directions
    
    
    m = Map(center=(38, -98), zoom=4, scroll_wheel_zoom=True)
    ant_path = AntPath(locations=[start, end])
    m.add_layer(ant_path)
    measure = MeasureControl(position='topleft', active_color='red', primary_length_unit='miles')
    m.add_control(measure)
    display(m)
    
    return distance.distance(start, end).miles




# directions
route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

# map
map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)

# add geojson to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# display map
map_directions





# interaction between function and widget
ipywidgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

# coordinates from Nashville, TN (-86.781247, 36.163532) to Miami, FL (-80.191850, 25.771645)
# order for coordinates is [lon, lat]
coordinates = [[-86.781247, 36.163532], [-80.191850, 25.771645]]

# directions
route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

# map
map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)

# add geojson to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# display map
map_directions